In [46]:
# Import requested libraries.
# Influxdb > quering InfluxDB database.
# Pandas > loading ".csv" files and handling data.
import os

from influxdb import InfluxDBClient
import pandas as pd

In [47]:
# Define constants for InfluxDB connection.
INFLUXDB_HOST = 'localhost'
INFLUXDB_PORT = 8086
INFLUXDB_DATABASE = 'mydb'
INFLUXDB_USERNAME = 'root'
INFLUXDB_PASSWORD = 'root'
INFLUXDB_SSL = False
INFLUXDB_VERIFY_SSL = False
INFLUXDB_TIMEOUT = 5000

In [48]:
# Create an influx client to interact with database
client = InfluxDBClient(
    host=INFLUXDB_HOST, port=INFLUXDB_PORT,
    username=INFLUXDB_USERNAME, password=INFLUXDB_PASSWORD,
    ssl=INFLUXDB_SSL, verify_ssl=INFLUXDB_VERIFY_SSL,
    timeout=INFLUXDB_TIMEOUT)

In [49]:
# Create new database and connect to it
client.create_database(INFLUXDB_DATABASE)
client.switch_database(INFLUXDB_DATABASE)

In [50]:
# Get list of files to read
DIRPATH = './data/Measurements/'
filenames = os.listdir(DIRPATH)

In [51]:
# Loop over filenames
for filename in filenames:

    # Read csv file
    filepath = DIRPATH + filename
    # parser = pd.datetime.fromtimestamp
    data = pd.read_csv(
        filepath, index_col=[0], sep='\t',
        header=None, parse_dates=True)

    # Get Timeserie metadata from filename
    filename_without_extension = os.path.splitext(filename)[0]
    location, measurement = filename_without_extension.split('_', 1)
    
    # Prepare and write data to influx database
    serie = [
        {
            'time': row[0],
            'measurement': measurement,
            'fields': {
                'value': row[1][1],
            },
            'tags': {
                'location': location
            }
        }
        for row in data.iterrows()
    ]
    
    client.write_points(serie, time_precision='s')

In [52]:
# Request data to check if all OK
result = client.query("SELECT * FROM Temperature WHERE \"location\" = 'Room1'")
some_data = [point for point in result.get_points()]
some_data

[{'location': 'Room1', 'time': '2017-03-09T00:51:30Z', 'value': 19.53},
 {'location': 'Room1', 'time': '2017-03-09T03:32:04Z', 'value': 19.37},
 {'location': 'Room1', 'time': '2017-03-09T05:12:26Z', 'value': 19.53},
 {'location': 'Room1', 'time': '2017-03-09T05:22:30Z', 'value': 20},
 {'location': 'Room1', 'time': '2017-03-09T05:32:33Z', 'value': 20.31},
 {'location': 'Room1', 'time': '2017-03-09T05:42:37Z', 'value': 20.47},
 {'location': 'Room1', 'time': '2017-03-09T06:02:43Z', 'value': 20.63},
 {'location': 'Room1', 'time': '2017-03-09T06:22:50Z', 'value': 20.79},
 {'location': 'Room1', 'time': '2017-03-09T06:32:55Z', 'value': 20.47},
 {'location': 'Room1', 'time': '2017-03-09T06:43:00Z', 'value': 20.16},
 {'location': 'Room1', 'time': '2017-03-09T06:53:04Z', 'value': 20.47},
 {'location': 'Room1', 'time': '2017-03-09T07:03:10Z', 'value': 20.63},
 {'location': 'Room1', 'time': '2017-03-09T07:23:19Z', 'value': 20.79},
 {'location': 'Room1', 'time': '2017-03-09T07:42:59Z', 'value': 20.